GitHub repo for this: https://github.com/reuven/2024-09September-08-decorators

# Decorators

1. What are decorators?
2. Writing your first decorator
3. Outer function storage
4. Inputs and outputs
5. Decorators that take arguments
6. Nested decorators
7. Decorating classes
8. Writing decorators as classes

In [1]:
# let's assume that we have two simple functions, a and b, which return something very simple

def a():
    return f'A!\n'

def b():
    return f'B!\n'

In [2]:
print(a())
print(b())

A!

B!



In [3]:
# our company has mandated that we put dashed lines above and below everything we print
# one option is for us to rewrite the functions

lines = '-' * 60 + '\n'

def a():
    return f'{lines}A!\n{lines}'

def b():
    return f'{lines}B!\n{lines}'

print(a())
print(b())

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



In [ ]:
# the problem is that we will need to repeat this for every function we write!

# how can I rewrite this, such that I repeat myself as little as possible?
# option 1: write a function that takes our function as input, and returns the lines + our function's output

